### Obesity - Classification (Semester B)


In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from bokeh.plotting import figure, output_file, show
from bokeh.palettes import magma
import matplotlib.cm as cm 
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')


np.set_printoptions(precision=5)#show 5 number after decimal point 

In [2]:
df = pd.read_csv("ObesityDataSet_raw_and_data_sinthetic.csv")
df.head(3)

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,Female,21.0,1.62,64.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,0.0,1.0,no,Public_Transportation,Normal_Weight
1,Female,21.0,1.52,56.0,yes,no,3.0,3.0,Sometimes,yes,3.0,yes,3.0,0.0,Sometimes,Public_Transportation,Normal_Weight
2,Male,23.0,1.80,77.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,2.0,1.0,Frequently,Public_Transportation,Normal_Weight


בסמסטר הקודם הורדנו רק את עמודת המשקל מכיוון שייש לה קשר ישיר ל-בי-אמ-איי. בשיפור העבודה ננסה להוריד גם את עומדת הגובה שכן גם לה יש קשר חזק ל-בי-אמ-איי.
בסמסטר הקודם המודלים שלנו בעיקר התקשו להגדיר מהו אדם רגיל ומהו אדם עם אוביסטי סוג 3. 
ננסה להבין מה יוצר את הקושי וכימד נוכל לעזור למודלים שלנו לסווג בצורה טובה יותר.
בסמסטר הקודם, מדד הדיוק הגבוהה ביותר שהיה לנו הוא 81%. ננסה להשיג ציון טוב יותר או לפחות לנסות שמודלים שלנו לא יטעו בצורה קיצונית (כמו להגדיר אדם כאובוסטי סוג 3 כאשר בפועל הוא בתת משקל)


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2111 entries, 0 to 2110
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Gender                          2111 non-null   object 
 1   Age                             2111 non-null   float64
 2   Height                          2111 non-null   float64
 3   Weight                          2111 non-null   float64
 4   family_history_with_overweight  2111 non-null   object 
 5   FAVC                            2111 non-null   object 
 6   FCVC                            2111 non-null   float64
 7   NCP                             2111 non-null   float64
 8   CAEC                            2111 non-null   object 
 9   SMOKE                           2111 non-null   object 
 10  CH2O                            2111 non-null   float64
 11  SCC                             2111 non-null   object 
 12  FAF                             21

In [4]:
#Other than 'Age', 'Weight' and 'Height' there is no reason for the other numeric columns to be 'float'. We'll change it.
columns = ["NCP", "CH2O", "TUE", "FAF","FCVC"]
for col in columns:
    df[col] = round(df[col]).astype('int')

In [5]:
df_res = df.copy(deep=True)
df_res = pd.get_dummies(df, columns=['Gender','family_history_with_overweight','FAVC','SMOKE',
                                     'SCC','MTRANS','FCVC','NCP','CAEC','CH2O','FAF','TUE','CALC'])
df_res.head(3)

,Age,Height,Weight,NObeyesdad,Gender_Female,Gender_Male,family_history_with_overweight_no,family_history_with_overweight_yes,FAVC_no,FAVC_yes,...,FAF_1,FAF_2,FAF_3,TUE_0,TUE_1,TUE_2,CALC_Always,CALC_Frequently,CALC_Sometimes,CALC_no
0,21.0,1.62,64.0,Normal_Weight,1,0,0,1,1,0,...,0,0,0,0,1,0,0,0,0,1
1,21.0,1.52,56.0,Normal_Weight,1,0,0,1,1,0,...,0,0,1,1,0,0,0,0,1,0
2,23.0,1.80,77.0,Normal_Weight,0,1,0,1,1,0,...,0,1,0,0,1,0,0,1,0,0


In [6]:
df_res.drop(columns=['Weight','Height'],inplace=True)

In [7]:
from sklearn.model_selection import train_test_split
X = df_res.drop(columns=["NObeyesdad"])
y = df_res['NObeyesdad'] 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [8]:
from sklearn.preprocessing import MinMaxScaler
norm = MinMaxScaler()
X_train_norm = norm.fit_transform(X_train)
X_test_norm = norm.transform(X_test)

In [9]:
# Before we try real models, we will try a dummyClassifier. we will try to get high score than the dummyModel.
from sklearn.dummy import DummyClassifier
dummy_clf = DummyClassifier(strategy="most_frequent", random_state=0)
dummy_clf.fit(X_train_norm, y_train)
dummy_clf.predict(X_train_norm)
print("The DummyClassifier score is:" ,dummy_clf.score(X_test_norm, y_test))

The DummyClassifier score is: 0.1608832807570978


In [10]:
from sklearn.metrics import classification_report
from sklearn import metrics

In [11]:
#KNN
from sklearn.neighbors import KNeighborsClassifier
KNN = KNeighborsClassifier().fit(X_train_norm, y_train)
y_pred = KNN.predict(X_test_norm)
norm_preci = round(metrics.precision_score(y_test, y_pred, average="weighted"),3)            
norm_accuracy = round(metrics.accuracy_score(y_test, y_pred),3)
print("precision_score:", norm_preci)
print("Accuracy:", norm_accuracy)
print("Classification Report: \n", metrics.classification_report(y_test, y_pred))

precision_score: 0.682
Accuracy: 0.686
Classification Report: 
                      precision    recall  f1-score   support

Insufficient_Weight       0.70      0.85      0.76        86
      Normal_Weight       0.62      0.32      0.43        93
     Obesity_Type_I       0.57      0.77      0.66       102
    Obesity_Type_II       0.70      0.86      0.78        88
   Obesity_Type_III       0.85      0.99      0.92        98
 Overweight_Level_I       0.64      0.51      0.57        88
Overweight_Level_II       0.69      0.44      0.54        79

           accuracy                           0.69       634
          macro avg       0.68      0.68      0.66       634
       weighted avg       0.68      0.69      0.67       634



In [12]:
from xgboost import XGBClassifier

XGB = XGBClassifier(objective="multi:softmax",num_class= 7, random_state=42)
XGB.fit(X_train, y_train, eval_metric='auc')
y_pred_XGB = XGB.predict(X_test)

accuracy = metrics.accuracy_score(y_test, y_pred_XGB)
print("Accuracy:", accuracy)
print("Classification Report: \n", metrics.classification_report(y_test, y_pred_XGB))

Accuracy: 0.7697160883280757
Classification Report: 
                      precision    recall  f1-score   support

Insufficient_Weight       0.75      0.85      0.80        86
      Normal_Weight       0.60      0.60      0.60        93
     Obesity_Type_I       0.78      0.76      0.77       102
    Obesity_Type_II       0.86      0.90      0.88        88
   Obesity_Type_III       0.96      0.99      0.97        98
 Overweight_Level_I       0.69      0.62      0.65        88
Overweight_Level_II       0.71      0.63      0.67        79

           accuracy                           0.77       634
          macro avg       0.76      0.77      0.76       634
       weighted avg       0.77      0.77      0.77       634



In [13]:

from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier 

DTC = DecisionTreeClassifier()
KNN = KNeighborsClassifier()
log_clf = LogisticRegression()

voting_clf = VotingClassifier(
 estimators=[('lr', log_clf),('knn',KNN),('dt', DTC)],
 voting='hard')

voting_clf.fit(X_train, y_train)


VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('knn', KNeighborsClassifier()),
                             ('dt', DecisionTreeClassifier())])

In [14]:
from sklearn.metrics import accuracy_score
for clf in (log_clf,  KNN,DTC, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))


LogisticRegression 0.580441640378549
KNeighborsClassifier 0.7145110410094637
DecisionTreeClassifier 0.6987381703470031
VotingClassifier 0.7160883280757098


In [15]:
#Searching for better performance out of the models with RandomizedSearch
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
#XGBClassifier
'''
parameters = {
    "colsample_bytree": [0.7, 0.3],
    "gamma": [0, 0.5],
    "learning_rate": [0.03, 0.3], # default 0.1 
    "max_depth": [2, 6], # default 3
    "n_estimators": [100, 150], # default 100
    "subsample": [0.6, 0.4]
}
'''
parameters = {
        'learning_rate': [ 0.1,0.3,0.03],
        'max_depth': [5,6,10],
        'min_child_weight': [1, 2],
        'subsample': [0.5,0.6,1],
        'colsample_bytree': [0.3,0.5, 0.7],
        'n_estimators' : [100,150,170,200]
         }

random_XGB = RandomizedSearchCV(XGBClassifier(),parameters, verbose=True, n_jobs=-1, 
                          cv=3,random_state=3, n_iter = 100, return_train_score=True)
random_XGB.fit(X_train,y_train, eval_metric='auc')
y_pred_XGB_random = random_XGB.predict(X_test)
print ("Accuracy Score = ",metrics.accuracy_score(y_test,y_pred_XGB_random))
print (random_XGB.best_params_)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
